# 사전 사용 패키지 다운로드

In [261]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

# konlpy 설치
# pip install konlpy

# Mecab 설치
# http://konlpy.org/en/latest/install/
# https://cleancode-ws.tistory.com/97

import konlpy
from konlpy.tag import Mecab

tokenizer = Mecab('C:\mecab\mecab-ko-dic') # setting tokenizer using Mecab()


pd.set_option('display.max_rows', 5000)

# 대회 데이터 다운로드 및 사전 확인

In [262]:
train = pd.read_csv("train.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기

# train.head(2)

In [263]:
# train.shape

In [264]:
# train['smishing'].value_counts()

In [265]:
# train[train['smishing'] == 1]

In [266]:
# test = pd.read_csv("public_test.csv")

# test.head(2)

In [267]:
# submission=pd.read_csv("submission_제출양식.csv")

# submission.head(2)

In [268]:
# train.shape, test.shape, submission.shape

총 train data의 행은 약 29만개, test data의 행은 약 1600개로 되어있다.

## train data under-sampling method

In [269]:
# Counter(train['smishing'])

In [270]:
# 19646/train.shape[0]

NLP 특성상 해당 train data를 모두 tokenizing 한 뒤, CountVectorizer를 하게 되면,

 colab 상에서는 과하게 ram을 잡아먹어 다운이 되는 경우가 생기기 때문에, 스미싱 비율에 맞게 under sampling을 수행하였다.

In [271]:
# 850/12600

#  smishing:850개  non-smishing :12600개       , 비율 0.06:0.94

In [272]:
# random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
# train_nsm_list=list(train[train['smishing']!=1].index)
# train_nsmishing=random.sample(train_nsm_list, 11750*7 )

# random.seed(2019)
# train_sm_list=list(train[train['smishing']==1].index)
# train_smishing=random.sample(train_sm_list, 850*7 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

# 성능 저하됨


In [273]:

# train 데이터내에서 다시 train: 70% , test : 30%  데이터 추출
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error

def get_train_test_split(X_train, y_train, test_size=0.3, random_state=0) :
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
                                       test_size=test_size, random_state=random_state)
    print('X_train:',X_train.shape)
    print('X_test:',X_test.shape)
    return  X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_train_test_split(train.iloc[:,:-1], train.iloc[:,-1], test_size=0.1, random_state=0)
X_train['smishing'] = y_train
# X_train.drop(['smishing'],axis=1,inplace =True)

X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)
# X_train
y_test.reset_index(drop=True,inplace=True)

X_train: (266350, 3)
X_test: (29595, 3)


In [274]:
y_test.value_counts()
# y_train
# type(y_test)

0    27774
1     1821
Name: smishing, dtype: int64

In [275]:
random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
train_nsm_list=list(X_train[X_train['smishing']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 11750)

random.seed(2019)
train_sm_list=list(X_train[X_train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 850 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

In [276]:
# random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
# train_nsm_list=list(train[train['smishing']!=1].index)
# train_nsmishing=random.sample(train_nsm_list, 11750 )

# random.seed(2019)
# train_sm_list=list(train[train['smishing']==1].index)
# train_smishing=random.sample(train_sm_list, 850 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

In [277]:
train_xx=X_train.iloc[train_smishing+train_nsmishing].reset_index(drop=True) #under sampling으로 나온 index들로 train data 선별

train_yy=DataFrame(X_train['smishing'],columns=['smishing']) 
train_yyy=train_yy.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)


In [278]:
# test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
# test_xx=DataFrame(test['text'])
# test_yyy=DataFrame(test['smishing'])
# train_xx.head(100)

In [279]:
X_test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_xx=X_test
test_yyy=DataFrame(y_test)
test_yyy

,smishing
0,0
1,0
2,0
3,0
4,1
...,...
29590,0
29591,1
29592,0
29593,0


In [280]:
train_xx.shape,train_yyy.shape,test_xx.shape,test_yyy.shape

((12600, 4), (12600, 1), (29595, 4), (29595, 1))

In [281]:
train_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( train_xx['text'], train_yyy['smishing'] ) )  ] # Mecab를 활용하여 text를 토큰화 시킴
test_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( test_xx['text'], test_yyy['smishing'] ) )  ]

12600it [00:02, 4877.20it/s]
29595it [00:06, 4827.38it/s]


In [282]:
stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]


필요없는 단어들을 제거하면서 동시에 모형에 사용하기 위한, X_train,test, Y_train,test를 생성한다

In [283]:
X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_train.append(" ".join(temp))

In [284]:
X_test = []
for lwords in test_doc:
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_test.append(" ".join(temp))

CountVecotrizer를 사용하여서 학습 데이터를 바탕으로 벡터화를 시키고 이를 test data에 적용한다

In [285]:
# v=CountVectorizer()

# v.fit(X_train)

# vec_x_train= v.transform(X_train).toarray()
# vec_x_test= v.transform(X_test).toarray()
#  TfidfVectorizer로 대체

In [286]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)

vec_x_train= tfidf_vect.transform(X_train).toarray()
vec_x_test= tfidf_vect.transform(X_test).toarray()
# 0.833643058 으로 향상


In [287]:
vec_x_train.shape, vec_x_test.shape

((12600, 12976), (29595, 12976))

naive bayes중 MultinomialNB를 사용하여서 학습 모형을 만들고 이를 test data에 적용한다

In [288]:
# m1= MultinomialNB()
# m1.fit(vec_x_train,Y_train)

In [289]:
# from sklearn.linear_model import LogisticRegression
# m1 = LogisticRegression()
# m1.fit(vec_x_train,Y_train)

In [290]:
from sklearn.model_selection import GridSearchCV

lr_clf = LogisticRegression()
# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정. 
params = { 'C':[0.01, 0.1, 1, 5, 10]}
m1 = GridSearchCV(lr_clf ,param_grid=params , cv=3 , scoring='accuracy' , verbose=1 )
m1.fit(vec_x_train , Y_train)
print('Logistic Regression best C parameter :',m1.best_params_ )

# # 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
# pred = grid_cv_lr.predict(X_test_tfidf_vect)
# print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Logistic Regression best C parameter : {'C': 5}


In [291]:
y_test_pred1=m1.predict_proba(vec_x_test)

In [292]:
y_test_pred1_one= [ i[1]  for i in y_test_pred1]

for i in range(len(y_test_pred1_one)) :
    if y_test_pred1_one[i] > 0.5:
        y_test_pred1_one[i] = 1
    else :
         y_test_pred1_one[i] = 0

y_test_pred1       

array([[9.98436388e-01, 1.56361174e-03],
       [9.99484024e-01, 5.15976289e-04],
       [9.90579760e-01, 9.42024003e-03],
       ...,
       [9.99757474e-01, 2.42525812e-04],
       [9.97261816e-01, 2.73818381e-03],
       [9.95606006e-01, 4.39399434e-03]])

In [293]:
# 정확도 평가

preds = y_test_pred1_one
y_test = test_yyy
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import f1_score,roc_auc_score
# from sklearn.metrics import classification_report

# accurqacy
accuracy = accuracy_score(y_test,preds)

print('accuracy:',accuracy)


# roc_auc
roc_auc = roc_auc_score(y_test,preds)

print('roc_auc:',roc_auc)
# preds

accuracy: 0.9994931576279777
roc_auc: 0.9958813838550247


In [294]:
len(preds),len(y_test)

(29595, 29595)

In [192]:
# y_test_pred1=m1.predict_proba(vec_x_test)
# y_test_pred1_one= [ i[1]  for i in y_test_pred1]

In [295]:
test = pd.read_csv("public_test.csv")

test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_xx=DataFrame(test['text'])
test_yyy=DataFrame(test['smishing'])


test_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( test_xx['text'], test_yyy['smishing'] ) )  ]

1626it [00:00, 3049.04it/s]


In [296]:
X_test = []
for lwords in test_doc:
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_test.append(" ".join(temp))

In [297]:
vec_x_test= v.transform(X_test).toarray()

In [298]:
y_test_pred1=m1.predict_proba(vec_x_test)
y_test_pred1_one= [ i[1]  for i in y_test_pred1]

test data에서 각 text당 스미싱 확률을 구할 수 있으니 이를 제출양식의 smishing 변수에 넣고 csv 파일로 내보낸다

In [299]:
submission=pd.read_csv("submission_제출양식.csv")
submission['smishing'] = y_test_pred1_one

submission.to_csv("2019-12-10-Smishing_0313_Logistic_Grid.csv",index=False) 
#현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

#  2019-12-06-Smishing_12_12 --->   train roc_auc: 0.9997338673341677,     test score:  0.73841      52위
#  2019-12-10-Smishing_0251  --->                                         test score :  0.833643058  59위

이제 csv 파일로 나온 답지 파일을 14회 대회 제출 게시판을 통해 제출하시면 됩니다